In [ ]:
%cd /Users/gokhankocmarli/Projects/bitirme_tezi/div2k_realesrgan_hat_trainer/

# Preperation of the Environment & Dataset

**Dowload the original HAT codebase.**

In [ ]:
!git clone https://github.com/XPixelGroup/HAT.git .codebases/HAT

**Install dependencies.**

In [ ]:
%pip install torch torchvision torchaudio
%pip install -r .codebases/HAT/requirements.txt
%cd .codebases/HAT/
!python setup.py develop
%cd ../..

**Download dataset DIV2K and unzip it.**

In [ ]:
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_HR.zip -P .datasets/DIV2K
!unzip .datasets/DIV2K/DIV2K_train_HR.zip -d .datasets/DIV2K
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_HR.zip -P .datasets/DIV2K
!unzip .datasets/DIV2K/DIV2K_valid_HR.zip -d .datasets/DIV2K
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_train_LR_bicubic_X2.zip -P .datasets/DIV2K
!unzip .datasets/DIV2K/DIV2K_train_LR_bicubic_X2.zip -d .datasets/DIV2K
!wget -c http://data.vision.ee.ethz.ch/cvl/DIV2K/DIV2K_valid_LR_bicubic_X2.zip -P .datasets/DIV2K
!unzip .datasets/DIV2K/DIV2K_valid_LR_bicubic_X2.zip -d .datasets/DIV2K


**Rename the directories.**

In [ ]:
!mv .datasets/DIV2K/DIV2K_train_HR .datasets/DIV2K/train_hr
!mv .datasets/DIV2K/DIV2K_valid_HR .datasets/DIV2K/valid_hr
!mv .datasets/DIV2K/DIV2K_train_LR_bicubic/X2 .datasets/DIV2K/train_lr
!mv .datasets/DIV2K/DIV2K_valid_LR_bicubic/X2 .datasets/DIV2K/valid_lr
!rm -rf .datasets/DIV2K/DIV2K_train_LR_bicubic
!rm -rf .datasets/DIV2K/DIV2K_valid_LR_bicubic

**Get the mutliscale converter & sub-image maker scripts from Real-ESRGAN codebase.**

In [ ]:
!mkdir .scripts
!wget https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/scripts/generate_multiscale_DF2K.py -P .scripts
!wget https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/scripts/extract_subimages.py -P .scripts
!wget https://raw.githubusercontent.com/xinntao/Real-ESRGAN/master/scripts/generate_meta_info.py -P .scripts

**Generate sub-images.**

In [ ]:
!python .scripts/extract_subimages.py --input .datasets/DIV2K/train_hr --output .datasets/DIV2K/sub_train_hr --crop_size 400 --step 200
!python .scripts/extract_subimages.py --input .datasets/DIV2K/valid_hr --output .datasets/DIV2K/sub_valid_hr --crop_size 400 --step 200
!python .scripts/extract_subimages.py --input .datasets/DIV2K/train_lr --output .datasets/DIV2K/sub_train_lr --crop_size 200 --step 100
!python .scripts/extract_subimages.py --input .datasets/DIV2K/valid_lr --output .datasets/DIV2K/sub_valid_lr --crop_size 200 --step 100

In [ ]:
# Remove the original images.
!rm -r .datasets/DIV2K/train_hr
!rm -r .datasets/DIV2K/valid_hr
!rm -r .datasets/DIV2K/train_lr
!rm -r .datasets/DIV2K/valid_lr

**Check the size of the dataset.**

In [ ]:
import os

# Check the sizes of the datasets' image count.
EQUALITY_CHECK = {
    "sub_train_hr": "sub_train_lr",
    "sub_valid_hr": "sub_valid_lr",
}

for hr, lr in EQUALITY_CHECK.items():
    hr_path = os.path.join(".datasets/DIV2K", hr)
    lr_path = os.path.join(".datasets/DIV2K", lr)
    hr_images = os.listdir(hr_path)
    lr_images = os.listdir(lr_path)
    if len(hr_images) != len(lr_images):
        print(f"[ERROR] Number of images in {hr} and {lr} are not equal.")
    print(f"[<OK>] Number of images in {hr} and {lr} are equal.")


# Check the total disk usage.
!echo "\nTotal Disk Usage"
!du -sh .datasets/DIV2K/

# Select a Subset of Whole Dataset for Faster Training

In [2]:
# Change these variables to your own dataset location.
METADATAS = {
    "train": {
        'root_location': '.datasets/DIV2K/',
        'dataset_location': '.datasets/DIV2K/sub_train_hr/',
        'meta_data_location': '.metadatas/DIV2K/meta_info_train.txt'
    },
    "valid": {
        'root_location': '.datasets/DIV2K/',
        'dataset_location': '.datasets/DIV2K/sub_valid_hr/',
        'meta_data_location': '.metadatas/DIV2K/meta_info_valid.txt'
    }
}

**Generate metadata files.**

In [8]:
from glob import glob
from os.path import join, relpath
from os import makedirs
from cv2 import imread
from numpy.random import rand

# Create a folder for metadata.
makedirs('.metadatas/DIV2K/', exist_ok=True)

# Select 10% of each dataset for faster training, and place them into metadata.
for label, meta_data in METADATAS.items():
    with open(meta_data['meta_data_location'], 'w') as meta_info_file:
        # Get all image paths.
        all_files = sorted(glob(join(meta_data['dataset_location'], '*')))
        
        # Traverse all images.
        for image_path in all_files:

            # Select a random number, if it is greater than 0.2, skip this image.
            if rand() > 0.50:
                continue

            # Check the image.
            try:
                image = imread(image_path)
            except (IOError, OSError) as error:
                print(f'Read {image_path} error: {error}')
                continue
            if image is None:
                print(f'Img is None: {image_path}')
                continue

            # Get the relative path.
            img_name = relpath(image_path, meta_data['root_location'])
            
            # Write into metadata.
            meta_info_file.write(f'{img_name}\n')

**The data length for validation and training sets.**

In [ ]:
# Calculate the line count of each metadata.
for label, meta_data in METADATAS.items():
    with open(meta_data['meta_data_location'], 'r') as meta_info_file:
        print(f'{meta_data["meta_data_location"]} has {len(meta_info_file.readlines())} lines.')

# Generate Configuration File for Training

**Copy a new configuration file from the original one.**

In [41]:
!mkdir -p .options/train
!cp .codebases/HAT/options/train/train_HAT-S_SRx2_from_scratch.yml .options/train/hat_2x_div2k.yml

In [48]:
# Read the configuration YAML file and change the parameters needed.
import yaml

with open('.options/train/hat_2x_div2k.yml', 'r') as file:
    config = yaml.safe_load(file)

if config['datasets']['train']['name'] != "DIV2K_Training_":
    config['datasets']['train']['name'] = "DIV2K_Training"
    config['datasets']['train']['dataroot_gt'] = ".datasets/DIV2K/"
    config['datasets']['train']['dataroot_lq'] = ".datasets/DIV2K/sub_train_lr" # It needs sub-dir as well.
    config['datasets']['train']['meta_info_file'] = ".datasets/DIV2K/meta_info_train.txt"
    config['datasets']['val']['name'] = "DIV2K_Validation"
    config['datasets']['val']['dataroot_gt'] = ".datasets/DIV2K/"
    config['datasets']['val']['dataroot_lq'] = ".datasets/DIV2K/sub_valid_lr"  # It needs sub-dir as well.
    config['datasets']['val']['meta_info_file'] = ".datasets/DIV2K/meta_info_valid.txt"

    # Remove unused validation.
    if 'val_2' in config['datasets']:
        del config['datasets']['val_2']
    if 'val_3' in config['datasets']:
        del config['datasets']['val_3']

    # Enable auto-resume has to be done using options file.
    config['auto_resume'] = True
    config['logger']['print_freq'] = 50
    config['logger']['save_checkpoint_freq'] = 1000.0

    # Change the total iteration.
    config['train']['total_iter'] = 25000

    # Save the configuration YAML file.
    with open('.options/train/hat_2x_div2k.yml', 'w') as file:
        yaml.dump(config, file, sort_keys=False)

# Demo Training (Short Period & w/ Debugging)

In [ ]:
!python .codebases/HAT/hat/train.py -opt .options/train/hat_2x_div2k.yml --debug

# Train!

In [ ]:
!python .codebases/HAT/hat/train.py -opt .options/train/hat_2x_div2k.yml --auto_resume